In [138]:
import pandas as pd
import numpy as np

import json

In [139]:
# load data
df = pd.read_hdf('sat-data.h5')
df

,NORAD_CAT_ID,EPOCH,SEMIMAJOR_AXIS,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY
0,41469,2024-03-29 23:14:42,42162.865,0.0005158,3.2631,122.0675,309.1062,234.7614
1,22787,2024-03-29 22:56:42,42164.272,0.0005063,11.6348,7.7743,18.0968,174.8006
2,22787,2024-03-29 22:56:42,42164.272,0.0005063,11.6348,7.7743,18.0968,174.8006
3,41469,2024-03-29 21:49:27,42162.856,0.0005158,3.263,122.0702,309.1057,213.3868
4,38704,2024-03-29 21:39:33,41632.950,0.0210129,2.2449,62.3939,214.7218,144.2019
...,...,...,...,...,...,...,...,...
1551,33436,2024-03-01 01:37:31,42165.891,0.0001378,0.0211,124.6008,281.0132,157.3407
1552,33373,2024-03-01 01:08:34,42164.289,0.0001905,0.007,141.2321,196.4168,116.7912
1553,44457,2024-03-01 00:49:42,42164.775,0.0000261,0.0133,127.4057,303.8112,112.5159
1554,22787,2024-03-01 00:41:42,42163.703,0.0003622,11.6257,7.9197,345.0296,205.4834


In [140]:
convert_dict = {
    "NORAD_CAT_ID": "object",
    "EPOCH": "datetime64",
    "SEMIMAJOR_AXIS": "float64",
    "ECCENTRICITY": "float64",
    "INCLINATION": "float64",
    "RA_OF_ASC_NODE": "float64",
    "ARG_OF_PERICENTER": "float64",
    "MEAN_ANOMALY": "float64"
}

# Convert columns using the dictionary
df = df.astype(convert_dict, errors='ignore')

# Print the data types of all columns
print(df.dtypes)

NORAD_CAT_ID                 object
EPOCH                datetime64[ns]
SEMIMAJOR_AXIS              float64
ECCENTRICITY                float64
INCLINATION                 float64
RA_OF_ASC_NODE              float64
ARG_OF_PERICENTER           float64
MEAN_ANOMALY                float64
dtype: object


In [141]:
df[df['NORAD_CAT_ID'] == '8832']

,NORAD_CAT_ID,EPOCH,SEMIMAJOR_AXIS,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY
8,8832,2024-03-29 21:34:21,42738.661,0.013807,20.1599,52.9431,74.0489,287.1113
43,8832,2024-03-29 09:34:28,42738.651,0.013807,20.1588,52.9495,74.0388,110.2720
44,8832,2024-03-29 09:34:28,42738.651,0.013807,20.1588,52.9495,74.0388,110.2720
69,8832,2024-03-28 21:32:56,42738.632,0.013805,20.1577,52.9558,74.0287,293.0323
105,8832,2024-03-28 08:33:27,42738.600,0.013805,20.1565,52.9626,74.0186,101.5567
154,8832,2024-03-27 08:31:42,42738.543,0.013802,20.1542,52.9753,74.0003,107.3985
155,8832,2024-03-27 08:31:42,42738.543,0.013802,20.1542,52.9753,74.0003,107.3985
193,8832,2024-03-26 19:47:34,42738.505,0.013800,20.1531,52.9820,73.9858,279.6966
218,8832,2024-03-26 08:16:12,42738.457,0.013799,20.1520,52.9880,73.9786,109.8658
242,8832,2024-03-25 19:46:07,42738.416,0.013796,20.1508,52.9946,73.9679,285.6073


In [142]:
def deduplicate_by_date(
    data: pd.DataFrame,
    group_by_id_col: str,
    date_col: str,
    sort_ascending: bool = False,
) -> pd.DataFrame:
    """
    Deduplicates data by keeping the most recent entry for each group based on date.

    Args:
        data: A pandas DataFrame containing the data.
        group_by_id_col: The column to group the data by (e.g., satellite ID).
        date_col: The column containing the date information for sorting and deduplication.
        sort_ascending: Whether to sort in ascending order (False for most recent first).

    Returns:
        A pandas DataFrame containing the deduplicated data (copy of the input).
    """

    # Operate on a copy to avoid modifying original data
    data_copy = data.copy()

    # Extract the date part from the date column (assuming format YYYY-MM-DD)
    data_copy['date'] = pd.to_datetime(data_copy[date_col]).dt.date

    return (
        data_copy.groupby([group_by_id_col, 'date'])
        .apply(lambda x: x.sort_values(date_col, ascending=sort_ascending).head(1))
        .drop(columns='date')
        .reset_index(drop=True)  # Restructure for consistency
    )

# Example usage
deduplicated_data = deduplicate_by_date(df.copy(), 'NORAD_CAT_ID', 'EPOCH', sort_ascending=False)


In [143]:
deduplicated_data[deduplicated_data['NORAD_CAT_ID'] == '8832']

,NORAD_CAT_ID,EPOCH,SEMIMAJOR_AXIS,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY
678,8832,2024-03-01 21:29:42,42738.513,0.013743,20.0977,53.2991,73.4786,101.5024
679,8832,2024-03-02 22:41:13,42738.631,0.013744,20.1000,53.2856,73.4944,112.8025
680,8832,2024-03-03 20:48:48,42738.713,0.013744,20.1021,53.2737,73.5083,78.9187
681,8832,2024-03-04 10:42:45,42738.744,0.013744,20.1034,53.2663,73.5184,283.7773
682,8832,2024-03-05 18:25:01,42738.779,0.013747,20.1063,53.2493,73.5321,31.0683
683,8832,2024-03-06 12:28:24,42738.766,0.013748,20.1080,53.2397,73.5446,297.1962
684,8832,2024-03-07 12:30:00,42738.741,0.013750,20.1102,53.2269,73.5558,291.3244
685,8832,2024-03-08 12:31:25,42738.667,0.013753,20.1124,53.2142,73.5675,285.4088
686,8832,2024-03-09 12:32:58,42738.575,0.013755,20.1146,53.2015,73.5817,279.5215
687,8832,2024-03-10 00:34:34,42738.525,0.013756,20.1156,53.1951,73.5868,96.7832


In [146]:
deduplicated_data

,NORAD_CAT_ID,EPOCH,SEMIMAJOR_AXIS,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY
0,20253,2024-03-01 17:36:02,42163.595,0.000675,12.6674,356.2155,349.1208,204.5277
1,20253,2024-03-02 16:55:42,42163.733,0.000677,12.6671,356.2095,349.4270,195.1141
2,20253,2024-03-03 19:50:04,42163.895,0.000681,12.6668,356.2024,350.1068,239.1494
3,20253,2024-03-04 16:39:42,42164.019,0.000682,12.6666,356.1970,350.5596,191.9705
4,20253,2024-03-05 04:48:42,42164.093,0.000683,12.6664,356.1939,350.8786,14.4059
...,...,...,...,...,...,...,...,...
697,8832,2024-03-25 19:46:07,42738.416,0.013796,20.1508,52.9946,73.9679,285.6073
698,8832,2024-03-26 19:47:34,42738.505,0.013800,20.1531,52.9820,73.9858,279.6966
699,8832,2024-03-27 08:31:42,42738.543,0.013802,20.1542,52.9753,74.0003,107.3985
700,8832,2024-03-28 21:32:56,42738.632,0.013805,20.1577,52.9558,74.0287,293.0323


In [191]:
def reshape_by_ws(data: pd.DataFrame, w: int = 20, step: int = 1) -> pd.DataFrame:
  """
  Reshapes time series data into segments using the Windowing Sliding (WS) method.

  Args:
    data: A pandas DataFrame containing the time series data with columns:
      - NORAD_CAT_ID: The NORAD Catalog ID (string).
      - EPOCH: The epoch time (datetime).
      - SEMIMAJOR_AXIS, ECCENTRICITY, INCLINATION, RA_OF_ASC_NODE, ARG_OF_PERICENTER, MEAN_ANOMALY: Orbital elements (numerical).
    w: The slicing width (window size) of each segment.
    step: The slicing step length for the window sliding (default=1).

  Returns:
    A pandas DataFrame containing the reshaped data with segments as rows. Each segment
    contains columns for NORAD_CAT_ID, all orbital elements, and a segment ID.

  Raises:
    ValueError: If the slicing width (w) is less than 1.
  """

  if w < 1:
      raise ValueError("Slicing width (w) must be greater than or equal to 1")

  def segment_data(group):
    """Segments data for a single NORAD_CAT_ID group."""
    # Sort by epoch (ensure data is ordered chronologically)
    group = group.sort_values(by='EPOCH')

    n = len(group)
    # Calculate number of segments (s) using integer division
    s = n // w

    # Create list to store segments with segment ID
    segments = []
    for i in range(s):
      start_idx = i * step
      end_idx = min(start_idx + w, n)

      # Extract segment data and create segment ID
      # segment_id = f"Segment_{i+1}"
      segment = group[['NORAD_CAT_ID'] + list(group.columns[1:])].iloc[start_idx:end_idx]
      # segment['segment_id'] = segment_id

      segments.append(segment)

    return pd.concat(segments)

  # 1. Group data by NORAD_CAT_ID
  grouped_data = data.groupby('NORAD_CAT_ID')

  # 2. Filter groups with less than window size (w) data points
  filtered_data = grouped_data.filter(lambda x: len(x) >= w)

  # group the filtered data
  grouped_filtered_data = filtered_data.groupby('NORAD_CAT_ID')

  # 3. Apply segment_data function to each filtered group
  reshaped_data = grouped_filtered_data.apply(segment_data)

  # drop segment_id column
  return reshaped_data

reshaped_data = reshape_by_ws(deduplicated_data, w=20, step=1)


In [197]:
reshaped_data[reshaped_data['NORAD_CAT_ID'] == '8832']

NORAD_CAT_ID               EPOCH  SEMIMAJOR_AXIS  \
NORAD_CAT_ID                                                        
8832         678         8832 2024-03-01 21:29:42       42738.513   
             679         8832 2024-03-02 22:41:13       42738.631   
             680         8832 2024-03-03 20:48:48       42738.713   
             681         8832 2024-03-04 10:42:45       42738.744   
             682         8832 2024-03-05 18:25:01       42738.779   
             683         8832 2024-03-06 12:28:24       42738.766   
             684         8832 2024-03-07 12:30:00       42738.741   
             685         8832 2024-03-08 12:31:25       42738.667   
             686         8832 2024-03-09 12:32:58       42738.575   
             687         8832 2024-03-10 00:34:34       42738.525   
             688         8832 2024-03-11 14:18:36       42738.323   
             689         8832 2024-03-12 14:20:14       42738.208   
             690         8832 2024-03-13 14:21:39       42738.095   
             691         8832 2024-03-14 02:23:00       42738.039   
             692         8832 2024-03-16 16:14:42       42737.857   
             693         8832 2024-03-17 16:09:42       42737.845   
             694         8832 2024-03-18 04:11:16       42737.839   
             695         8832 2024-03-20 17:56:35       42737.940   
             696         8832 2024-03-22 17:59:25       42738.113   
             697         8832 2024-03-25 19:46:07       42738.416   

                  ECCENTRICITY  INCLINATION  RA_OF_ASC_NODE  \
NORAD_CAT_ID                                                  
8832         678      0.013743      20.0977         53.2991   
             679      0.013744      20.1000         53.2856   
             680      0.013744      20.1021         53.2737   
             681      0.013744      20.1034         53.2663   
             682      0.013747      20.1063         53.2493   
             683      0.013748      20.1080         53.2397   
             684      0.013750      20.1102         53.2269   
             685      0.013753      20.1124         53.2142   
             686      0.013755      20.1146         53.2015   
             687      0.013756      20.1156         53.1951   
             688      0.013761      20.1191         53.1751   
             689      0.013763      20.1212         53.1625   
             690      0.013767      20.1234         53.1498   
             691      0.013768      20.1245         53.1435   
             692      0.013776      20.1301         53.1107   
             693      0.013779      20.1323         53.0980   
             694      0.013780      20.1334         53.0916   
             695      0.013787      20.1392         53.0589   
             696      0.013791      20.1438         53.0335   
             697      0.013796      20.1508         52.9946   

                  ARG_OF_PERICENTER  MEAN_ANOMALY  
NORAD_CAT_ID                                       
8832         678            73.4786      101.5024  
             679            73.4944      112.8025  
             680            73.5083       78.9187  
             681            73.5184      283.7773  
             682            73.5321       31.0683  
             683            73.5446      297.1962  
             684            73.5558      291.3244  
             685            73.5675      285.4088  
             686            73.5817      279.5215  
             687            73.5868       96.7832  
             688            73.6126      292.9427  
             689            73.6319      287.0736  
             690            73.6513      281.1560  
             691            73.6585       98.3575  
             692            73.7230      290.1271  
             693            73.7483      282.6266  
             694            73.7593       99.8833  
             695            73.8305      290.0745  
             696            73.8859      278.2242  
             697   